# GBK->UTF8 (Don't run)

In [ ]:
# import os

# rootdir = 'data/'
# if os.path.exists('utf-8'):
#   os.system('rm -rf utf-8')
# for subdir, dirs, files in os.walk(rootdir):
#   for i,file in enumerate(files):
#       orignal_filename = os.path.join(subdir, file)
#       new_filename = os.path.join('\'utf-8',orignal_filename)+'\''
#       #new_filename = os.path.join('utf-8',subdir, str(i)+".txt")
#       os.system("mkdir -p `dirname "+new_filename+"`")
#       os.system("iconv -c -f GB2312 -t UTF-8 \'"+orignal_filename+'\' > '+new_filename)

# Annoated->Json

In [6]:
import copy
import json
import re,os
rootdir = b'utf-8/data/people/annotated'
parse_1 = re.compile(r'\$<*(.+)\[')
parse_1_1 = re.compile(r'\$<*(.+)=')
parse_1_2 = re.compile(r'\$<*(.+)>\$')
parse_2 = re.compile(r'\[(.+)\]')
parse_3 = re.compile(r'=(.*)>')
parse_4 = re.compile(r'<(.*?)>')
parse_5 = re.compile(r'\$<(.*?)>\$')
parse_6 = re.compile(r'#<(.*?)>#@')
passages = []
def removeEmpty(list_in):
  list_out = []
  for i in list_in:
    if i.strip()!='':
      list_out.append(i.strip())
  if list_out == []:
    list_out.append("NULL")
  return list_out

def removeNum(list_in):
  list_out = []
  for i in list_in:
    seg = i.split("<")
    seg = [re.sub(r'[0-9#$<>\s]',"",x) for x in seg]
    list_out.extend(seg)
  return list_out
  if list_out == []:
    list_out.append("NULL")
  return list_out
for subdir, dirs, files in os.walk(rootdir):
  for ttt,file in enumerate(files):
      paragraphs = []
      paragraph = {}
      paragraph_groups = []
      sentences = []
      passage = {}
      
      filename = os.path.join(subdir, file)
      f = open(filename,'r',encoding='utf-8')
      first_line = f.readline().strip()
      while first_line == "":
        first_line = f.readline().strip()
      try:
        if '[' not in first_line:
          gender = "NULL"
        else:
          gender = re.findall(parse_2,first_line)[0]
          
        if '=' not in first_line:
          qualities = ["NULL"]
        else:
          qualities = re.findall(parse_3,first_line)[0].split("+")
        if gender == "NULL":
          if qualities ==["NULL"]:
            figure = re.findall(parse_1_2,first_line)[0]
          else:
            figure = re.findall(parse_1_1,first_line)[0]
        else:
            figure = re.findall(parse_1,first_line)[0]
        second_line = f.readline().strip()
        while second_line == "":
          second_line = f.readline().strip()
        if "#" in second_line:
          sentiments = re.findall(parse_4,second_line)
          third_line = f.readline().strip()
          while third_line == "":
            third_line = f.readline().strip()
        else:
          third_line = second_line
          sentiments = ["NULL"]
        title = third_line.strip()
        fourth_line = f.readline().strip()
        while fourth_line == "":
            fourth_line = f.readline().strip()
        if "总" in fourth_line and "分" in fourth_line:
          structure = re.findall(parse_4,fourth_line)[0]
        else:
          structure = "NULL"
      except:
        print(filename)
        continue
      qualities = removeEmpty(qualities)
      sentiments = removeEmpty(sentiments)
      passage['figure'] = figure.strip() #if "陌生人[" not in p['figure'] else "陌生人"
      passage['gender'] = gender.strip()   #if '[' not in p['gender'] else p['gender'][-1]
      passage['qualities'] = qualities
      passage['sentiments'] = sentiments
      passage['title'] = title.strip()
      passage['structure'] = structure.strip()
      
      if structure == "NULL":
        passage_first_line = fourth_line
      else:
        passage_first_line = f.readline()
      para_info = re.findall(parse_5,passage_first_line)
      para_info = removeEmpty(para_info)
      para_info = removeNum(para_info)
      paragraph_group = {"paraGroupFunc":para_info}
      para_index = 0
      for index,line in enumerate(f):
        para_info = re.findall(parse_5,line)
        if para_info:
          if paragraphs:
            paragraph_group['paraGroup'] = paragraphs
            paragraph_groups.append(copy.deepcopy(paragraph_group))
          paragraphs = []
          para_info = removeEmpty(para_info)
          para_info = removeNum(para_info)
          paragraph_group = {"paraGroupFunc":para_info}
        else:
          sentences = []
          parse_add_and = re.compile(r"(#)[\u4e00-\u9fa5]")
          line = re.sub(parse_add_and,"#@",line)
          splits = re.split(parse_6,line.strip())
          annotations = re.findall(parse_6,line)
          sen_annotate = []
          for i in range(len(splits)):
            if splits[i] == "":
              continue
            if splits[i] not in annotations:
              splits[i] = splits[i].replace("。。。。。。","......")
              splits[i] = splits[i].replace("……","......")
              splits[i] = splits[i].replace("``````","......")
              splits[i] = splits[i].replace("”","\"")
              splits[i] = splits[i].replace("“","\"")
              splits[i] = splits[i].replace("＂","\"")
              splits[i] = splits[i].replace("DD","--")
              sentence = {"senGroupFunc":sen_annotate,"senGroup":splits[i].strip()}
              sentences.append(copy.deepcopy(sentence))
              sen_annotate = []
            else:
              sen_annotate.append(splits[i])
          if sentences == []:
            continue
          paragraph["paraIndex"] = para_index
          paragraph["paragraph"] = sentences
          para_index+=1
          paragraphs.append(copy.deepcopy(paragraph))
      paragraph_group['paraGroup'] = paragraphs
      paragraph_groups.append(copy.deepcopy(paragraph_group))
      passage['filename'] = filename.decode('utf-8')
      passage['paraGroups'] = paragraph_groups
      passages.append(passage)

def sgf_split(sgf_list,sen_list,current_index):
  output_list = []
  num_pattern = re.compile(r'([1-9])')
  for i in sgf_list:
    seg = i.split("<")
    seg = [re.sub(r'[#$<>\s]',"",x) for x in seg]
    for s in seg:
      para = re.sub('([^1-9])([1-9])', r"\1\n\2", s).split('\n')
      if len(para)==1:
        para.append(len(sen_list))

      para = (para[0],list(range(current_index,int(para[1])+current_index)))
      output_list.append(para)
      if len(para[1])<len(sen_list):
        after_para = (para[0]+"_after", list(range(para[1][-1]+1,current_index+len(sen_list))))
        output_list.append(after_para)
  return output_list
def cut_sent(para):
    para = re.sub('\?\?\?\?\?\?',"\?",para)
    para = re.sub('\(\s\)','',para)
    para = re.sub('([。！!?\?])([^\"”＂’\'。！!?\?])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('(\.{6})([^\"”＂’\'])', r"\1\n\2", para)  # 英文省略号
    para = re.sub('(\…{2})([^\"”＂’\'])', r"\1\n\2", para)  # 中文省略号
    para = re.sub('([。！!？\?][\"”＂’\'])([^，。！？\?])', r'\1\n\2', para)
    # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.rstrip()  # 段尾如果有多余的\n就去掉它
    # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
    return para.split("\n")

for ttt,p in enumerate(passages):
  sentences = []
  annotations = []
  paragraphs = []
  paragraph_groups = []
  for pg in p['paraGroups']:
    previous_paragraph_num = len(paragraphs)
    for para in pg['paraGroup']:
      previous_sentences_num = len(sentences)
      for sg in para['paragraph']:
        annotation = sg['senGroupFunc']
        sequence = sg['senGroup']
        sen_list = cut_sent(sequence)
        annotation_tuples = sgf_split(annotation,sen_list,len(sentences))
        annotations.extend(annotation_tuples)
        sentences.extend(sen_list)
      paragraphs.append((para['paraIndex'],list(range(previous_sentences_num,len(sentences)))))
    paragraph_groups.append((pg['paraGroupFunc'],list(range(previous_paragraph_num,len(paragraphs)))))
  # quota
  total_pair_quota = 0
  for i in range(len(sentences)-1):
    quota_count = len(re.findall("\"",sentences[i]))
    if quota_count==0:
      continue
    if quota_count%2!=0 and sentences[i][-1] == "\"" and total_pair_quota%2==0:
      sentences[i+1]="\""+sentences[i+1]
      sentences[i]=sentences[i][:-1]
      total_pair_quota+=quota_count-1
    else:
      total_pair_quota+=quota_count
  p['all_sentences'] = sentences
  p['pair_annotation_sentences'] = annotations 
  p['pair_paragraph_sentences'] = paragraphs
  p['pair_annotations_paragraphs'] = paragraph_groups
with open('people.json', 'w',encoding='utf-8') as fout:
  json.dump(passages,fout,indent=4,ensure_ascii=False)
     #fout.write(json.dumps(passages, indent=4,ensure_ascii=False).encode('utf8'))

b'utf-8/data/people/annotated/6-1/6_052\xe4\xbc\x9a\xe8\xaf\xb4\xe8\xaf\x9d\xe7\x9a\x84\xe7\x9c\xbc\xe7\x9d\x9b.txt'
b'utf-8/data/people/annotated/8-1/8_205\xe6\x88\x91\xe4\xb8\x8e\xe7\x88\xb6\xe4\xba\xb2.txt'
b'utf-8/data/people/annotated/9-1/9_105\xe5\xa4\x96\xe5\xa9\x86\xe7\x9a\x84\xe8\xae\xb0\xe5\xbf\x86-.txt'
b'utf-8/data/people/annotated/9-1/9_104\xe5\xa4\x96\xe5\xa9\x86\xe7\x9a\x84\xe6\x89\x8b.txt'
b'utf-8/data/people/annotated/9-1/9_553\xe8\xae\xa9\xe6\x88\x91\xe5\xbf\x83\xe5\xad\x98\xe6\x84\x9f\xe6\xbf\x80\xe7\x9a\x84\xe4\xba\xba3.txt'
b'utf-8/data/people/annotated/9-1/9_046\xe4\xbd\xa0\xe6\x98\xaf\xe6\x88\x91\xe5\xbf\x83\xe4\xb8\xad\xe7\x9a\x84\xe6\xad\x8c.txt'
b'utf-8/data/people/annotated/9-1/9_408\xe6\x9d\x91\xe9\x95\xbf\xe5\xb0\xbd\xe5\x9b\xb9\xe5\x9c\x84.txt'
b'utf-8/data/people/annotated/9-1/9_560\xe8\xae\xa9\xe6\x88\x91\xe8\xbd\xbb\xe8\xbd\xbb\xe5\x9c\xb0\xe5\x91\x8a\xe8\xaf\x89\xe4\xbd\xa0.txt'
b'utf-8/data/people/annotated/9-1/9_400\xe6\x9c\x8b\xe5\x8f\x8b (2).txt'


# Train/Val/Test Split

In [17]:
import json
import numpy as np
passages = json.load(open('people.json', 'r',encoding='utf-8'))
total_num = len(passages)
np.random.seed(42)

all_index = list(range(0,total_num))
np.random.shuffle(all_index)
train_index = all_index[:int(total_num*0.7)]
valid_index = all_index[int(total_num*0.7):int(total_num*0.8)]
test_index = all_index[int(total_num*0.8):]

In [18]:
train_text_file = open("processed_data/train.txt",'w',encoding='utf-8')
valid_text_file = open("processed_data/valid.txt",'w',encoding='utf-8')
test_text_file = open("processed_data/test.txt","w",encoding='utf-8')
for index,passage in enumerate(passages):
  if index in train_index:
    text_file = train_text_file 
  elif index in valid_index:
    text_file = valid_text_file
  else:
    text_file = test_text_file
  for pair_paragraph_sentences in passage['pair_paragraph_sentences']:
    for sentence_index in pair_paragraph_sentences[1]:
      text_file.write(passage['all_sentences'][sentence_index])
      text_file.write('\n')
    text_file.write('\n')
  text_file.write('\n')
train_text_file.close()
test_text_file.close()
valid_text_file.close()

random_poses = []
test_text_file = open("/data/baihe/dataset/Chinese_Eassy/processed_data/test.txt","r",encoding='utf-8')
test_input = open("/data/baihe/dataset/Chinese_Eassy/processed_data/test_partA.txt","w",encoding='utf-8')
test_output = open("/data/baihe/dataset/Chinese_Eassy/processed_data/test_partB.txt","w",encoding='utf-8')
text = test_text_file.read()
passages =text.strip().split('\n\n\n')
for index,passage in enumerate(passages):
  random_pos = np.random.randint(int(len(passage)*0.5))
  while len(passage[:random_pos].split('\n')) ==1 or passage[random_pos]=='\n' or passage[random_pos-1]!='\n':
    random_pos+=1
  random_poses.append(random_pos)
#   while passage[random_pos]=='\n':
#     random_pos-=1
  test_input.write(passage[:random_pos])
  test_input.write("\n"+"*"*10+str(index)+"*"*10+'\n')
  test_output.write(passage[random_pos:])
  test_output.write("\n"+"*"*10+str(index)+"*"*10+'\n')
test_input.close()
test_output.close()

# State Sequence

In [19]:
# State of Sentence
# State of Passage
# State of Paragraph
state = "paragraph"
start_para = ["总领全文","概括介绍","俗语开篇","诗歌开篇","歌词开篇","故事开篇","对话开篇","题记开篇","开门见山", "设置悬念","背景介绍"]
mid_para = ["过渡" ,"转折","中心段","点题照应", "人物描写" , "事件描写" ]
end_para = ["首尾照应","文题照应","前后照应","抒情","主题升华"]
para_dict = {x:"start" for x in start_para}
para_dict.update({x:"end" for x in end_para})
para_dict.update({x:"mid" for x in mid_para})

passages = json.load(open('people.json', 'r',encoding='utf-8'))
train_state_file = open("processed_data/state_para/train.txt",'w',encoding='utf-8')
valid_state_file = open("processed_data/state_para/valid.txt",'w',encoding='utf-8')
test_state_file = open("processed_data/state_para/test.txt","w",encoding='utf-8')
for index,passage in enumerate(passages):
  if index in train_index:
    state_file = train_state_file 
  elif index in valid_index:
    state_file = valid_state_file
  else:
    state_file = test_state_file
  
  for pair_annotations_paragraphs in passage['pair_annotations_paragraphs']:
    para_index_list = pair_annotations_paragraphs[1]
    para_func_list = pair_annotations_paragraphs[0]
    if para_func_list[0] not in para_dict.keys():
      continue
    for para_index in para_index_list: 
      sent_index_list = passage['pair_paragraph_sentences'][para_index][1]
      for sent_index in sent_index_list:
        state_file.write(passage['all_sentences'][sent_index].strip())
      state_file.write("|||")
      state_file.write(para_dict[para_func_list[0]].strip())
      #state_file.write(" ".join(para_func_list))
      state_file.write('\n')
  state_file.write('\n')
train_state_file.close()
test_state_file.close()
valid_state_file.close()

In [30]:
state = "figure"
figure_count = dict()
passages = json.load(open('people.json', 'r',encoding='utf-8'))
train_state_file = open("processed_data/figure/train.txt",'w',encoding='utf-8')
valid_state_file = open("processed_data/figure/valid.txt",'w',encoding='utf-8')
test_state_file = open("processed_data/figure/test.txt","w",encoding='utf-8')
for index,passage in enumerate(passages):
  if "[" in passage['figure']:
    passage['figure'] = passage['figure'][4:-1]
  if passage['figure'] in figure_count.keys():
    figure_count[passage['figure']]+=1
  else:
    figure_count[passage['figure']]=1
  if index in train_index:
    text_file = train_state_file 
  elif index in valid_index:
    text_file = valid_state_file
  else:
    text_file = test_state_file
  for pair_paragraph_sentences in passage['pair_paragraph_sentences']:
    for sentence_index in pair_paragraph_sentences[1]:
      text_file.write(passage['all_sentences'][sentence_index])
      text_file.write(" ||| "+passage['figure'])
      text_file.write('\n')
    text_file.write('\n')
  text_file.write('\n')
  
train_state_file.close()
test_state_file.close()
valid_state_file.close()

In [ ]:
state = "prompt"
passages = json.load(open('people.json', 'r',encoding='utf-8'))
train_state_file = open("processed_data/prompt/train.txt",'w',encoding='utf-8')
valid_state_file = open("processed_data/prompt/valid.txt",'w',encoding='utf-8')
test_state_file = open("processed_data/prompt/test.txt","w",encoding='utf-8')
for index,passage in enumerate(passages):
  if index in train_index:
    text_file = train_state_file 
  elif index in valid_index:
    text_file = valid_state_file
  else:
    text_file = test_state_file
  for pair_paragraph_sentences in passage['pair_paragraph_sentences']:
    for sentence_index in pair_paragraph_sentences[1]:
      text_file.write(passage['all_sentences'][sentence_index])
      text_file.write(" ||| "+passage['figure'])
      text_file.write('\n')
    text_file.write('\n')
  text_file.write('\n')
  
train_state_file.close()
test_state_file.close()
valid_state_file.close()

# Topic focus

In [46]:
f = open('processed_data/figure/test.txt','r',encoding='utf-8').read().split('\n\n\n')[:-1]

In [80]:
topics = ['母亲', '同桌', '姥爷', '老师', '老妈', '同学', '父亲',
          '姥姥', '哥哥', '清洁工', '弟弟', '爷爷', '陌生人', '奶奶',
          '姐姐', '阿姨', '偶然遇到的人', '环卫工人', '班主任', '外婆', '邻居',
          '妹妹', '乞丐', '表妹', '小偷', '伯父', '表弟', '爸爸', '表哥',
          '修鞋师傅', '叔叔', '妈妈', '老爸', '舅舅', '姑姑', '捡破烂奶奶',
          '外公',  '教练', '盲人', '老奶奶', '卖饼阿姨',
          '售票员', '教官', '乘客', '补鞋老人', '墓地少女', '老妪', '擦皮鞋小女孩',
          '交警', '邻居爷爷', '表姐', '婆婆', '川剧大师', '堂姐', '卖别针的老人', '少年',
          '邮递员', '老太太', '捡破烂老人', '磨剪刀爷爷', '修车师傅', '小女孩', '祖母',
          '老爷爷', '小摊主', '警察叔叔', '高师傅', '男老师', '太姥爷', '司机', '路人',
          '阿婆', '狱警', '村长', '售货员', '祖父', '乘务员', '拾荒者', '电梯师傅',
          '民工', '护士', '姨妈', '继父', '残疾人', '堂弟', '园丁', '运货员',
          '学姐', '前男友', '消防队员', '外祖母', '侄子', '闺蜜',
          '对手', '知己', '拾荒奶奶', '养父', '眼镜男', '姐妹', '太姥爷和太姥姥', '磨刀人',
          '卖馍老头', '老夫妇', '干爸', '农村小姑娘', '太姥姥', '师姐', '食堂大妈',
          '交通协管员', '小侄儿', '修车老爷爷', '太祖母', '买菜女', '校长', '废品老人']

In [81]:
f_out = open('processed_data/figure/test_topic_label.txt','w',encoding='utf-8')
f_all = open('processed_data/figure/all_topic.txt','w',encoding='utf-8')
for t in topics:
  f_all.write(t+'\n')
for line in f:
  annotation = line.split('\n')[0].split('|||')[1].strip()
  f_out.write(annotation+'\n')
f_out.close()
f_all.close()

In [94]:
import re 
pattern = r"\*" * 10 + "[0-9]+" + "\*" * 10
generated_text = open('processed_data/test_partB.txt','r',encoding='utf-8').read()
labels = open('processed_data/figure/test_topic_label.txt','r',encoding='utf-8').read().strip().split('\n') 
all_label = open('processed_data/figure/all_topic.txt','r',encoding='utf-8').read().strip().split('\n')
generated_passages = re.split(pattern, generated_text)[:-1]


In [95]:
target_per = []
count=0
for l,p in zip(labels,generated_passages):
  target_num = p.count(l)
  all_num = sum([p.count(x) for x in all_label])
  if all_num==0:
    count+=1
    continue
  target_per.append(target_num/all_num)


In [96]:
sum(target_per)/len(target_per)

0.45526175371479094

## Writing Prompts

In [1]:
from transformers import GPT2Tokenizer
import spacy 
import re
from tqdm import tnrange, tqdm_notebook
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path='gpt2',cache_dir="/data/baihe/dataset/gpt2")
nlp = spacy.blank("en")
nlp.add_pipe(nlp.create_pipe("sentencizer"))
def split_para(text):
  text = text.replace("<newline> <newline>","<newline>")
  paragraphs = text.split("<newline>")
  while "" in paragraphs:
    paragraphs.remove("")
  return paragraphs
def split_sent(text):
  sentences = [x.text.strip() for x in nlp(text).sents]
  return sentences
def wp_preprocess(text):
    # Standardize some symbols
    text = text.replace('``', '"')
    text = text.replace("''", '"')
    # Detokenize
    text = re.sub(' +', ' ', text)
    text = re.sub(' (\'|\.|\,|\:|\?|\!|;)', '\g<1>', text)
    text = re.sub('" (.*) "', '"\g<1>"', text)
    text = text.replace(" n't", "n't")
    return text


/home/baihe/.conda/envs/tf36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/baihe/.conda/envs/tf36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/baihe/.conda/envs/tf36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/baihe/.conda/envs/tf36/lib/python3.6/site-packages/tensorf

In [3]:
prefix = "/data/baihe/dataset/writingPrompts/original_data/"
data = ["train"]
for name in data:
  f_tar_out = open(prefix+name+".wp_target_unfiltered",'w',encoding='utf-8')
  f_src_out = open(prefix+name+".wp_source_unfiltered",'w',encoding='utf-8')
  with open(prefix+ name + ".wp_target",'r',encoding='utf-8') as f_1:
    with open(prefix+ name + ".wp_source",'r',encoding='utf-8') as f_2:
      src_all_data,tgt_all_data = f_2.readlines(),f_1.readlines()
      total_examples=0
      for source,target in tqdm_notebook(zip(src_all_data,tgt_all_data),total=len(src_all_data)):

        src_passage = wp_preprocess(re.sub('\[ (.*) \]', '', source))
        if src_passage=='':
          continue
        tgt_passage = wp_preprocess(target)
        source_paras = split_para(src_passage.strip())
        target_paras = split_para(tgt_passage.strip())
#         text = " ".join(source_paras+target_paras)
#         total_tokens= len(source_paras)+len(target_paras)+3+len(tokenizer.tokenize(text))+len(split_sent(text))
#         if total_tokens>1024:
#           continue
        total_examples+=1
        
        for para in source_paras:
          sents = split_sent(para)
          f_src_out.write(' [SENTENCE_EOS] '.join(sents)+' [SENTENCE_EOS]')
          f_src_out.write(' [PARAGRAPH_EOS] ')
        for para in target_paras:
          sents = split_sent(para)
          f_tar_out.write(' [SENTENCE_EOS] '.join(sents)+' [SENTENCE_EOS]')
          f_tar_out.write(' [PARAGRAPH_EOS] ')
        f_src_out.write('\n')
        f_tar_out.write('\n')
  print(total_examples)
  f_tar_out.close()
  f_src_out.close()


272600


In [12]:
text

"[ cw ] write about the strangest/scariest/saddest dream you 've ever had in less than 200 words . The night was as thick and terrifying as any I had ever seen before . All I could hear was the scream of the wind past my ears , the pounding of hooves , huffed horse breaths , and the pounding of my own heart .   The woods were closeknit , and my path was barely visible , hidden under a thick layer of bracken .   `` Faster , '' I whispered as I dug my heels in . Safety was close and yet so far away , calling to me . He would save me ; I knew it with all my heart .   All I had to do was outrun the demons at my back first ."

In [6]:
prefix = "/data/baihe/dataset/writingPrompts/original_data/"
data = ["valid","train", "test"]
for name in data:
  f_tar_out = open(prefix+name+".wp_target_filtered_w_newline",'w',encoding='utf-8')
  f_src_out = open(prefix+name+".wp_source_filtered_w_newline",'w',encoding='utf-8')
  with open(prefix+ name + ".wp_target",'r',encoding='utf-8') as f_1:
    with open(prefix+ name + ".wp_source",'r',encoding='utf-8') as f_2:
      src_all_data,tgt_all_data = f_2.readlines(),f_1.readlines()
      for source,target in tqdm_notebook(zip(src_all_data,tgt_all_data),total=len(src_all_data)):
        if len(tokenizer.tokenize(source+target))>1024:
          continue
        f_src_out.write(source)
        f_tar_out.write(target)
  f_tar_out.close()
  f_src_out.close()

In [4]:
prefix = "/data/baihe/dataset/writingPrompts/original_data/"
name='train'
with open(prefix+ name + ".wp_source",'r',encoding='utf-8') as f_2:
  src_all_data = f_2.readlines()
  length = []
  for source in tqdm_notebook(src_all_data,total=len(src_all_data)):
    length.append(len(tokenizer.tokenize(source)))
  print(sum(length)/len(length))



30.029831254585474


In [9]:
%matplotlib inline
import pandas as pd

In [11]:
df = pd.DataFrame({"length":length})

In [15]:
len(length)

272600

In [20]:
import torch

In [23]:
a=torch.tensor([1,2.3])

In [24]:
torch.mean(a)

tensor(1.6500)